<a href="https://colab.research.google.com/github/josephmailil1/Speciale---Stibo/blob/main/Agent%20Tool%20Evaluation%20Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing neassary packages

In [ ]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Storing OpenAI API key

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_Key') #use the name of your colab key

Importing Pandas for storing data

In [ ]:
import pandas as pd

Mouting to drive to access the uploaded csv. files fra shap script

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Setting Child directory to access the files

In [ ]:
import os
os.chdir("/content/drive/My Drive/Speciale")

## Load data

In [ ]:
shap = pd.read_csv("shap_noincome.csv")
data = pd.read_csv("basedata.csv")

Loading the LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, openai_api_key=openai_api_key)

In [ ]:
from langchain_experimental.tools import PythonAstREPLTool

Loading the PythonREPLtool

In [ ]:
tools = [PythonAstREPLTool(locals={"data": data, "shap": shap})]

Loading the PythonREPLtool

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain.output_parsers import JsonOutputToolsParser

In [ ]:
parser = JsonOutputToolsParser(tool = tools)

# Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""You have access to a pandas dataframe `data`. \
            This is the final dataframe that was used to predict Income with, using a Random Forest ML model. \
            The dataset is a publicly available dataset called `adult census`, where prediction task is to determine whether a person makes over $50K a year. \
             Here is the dataset for the first 5 obsevations

              ```
              {data.head().to_markdown()}
              ```

            You are also given the pandas dateframe `shap`. This dataframe contains shap values for the test set obsevations. \
            The shap obsevation shows which values are important for the final prediction of each local prediction.
            Here is the shap dataframe for the first 5 obsevations
             ```
            {shap.head().to_markdown()}
            ```
            If asked about the importance/relevance of obsevations or features, use the `shap` dataframe to answer. \
            A high absolute value of the shap dataset means that the value is relevant. \
            If a prediction is made on race, sex, or age please highlight the possibility that the individual is a victim of discrimination \

            Use the `data` dataframe to answer question about what the input values to the obsevation is. \
            Both Dataframe start at index 0.

            Respond directly to the question once you have enough information to answer it. \
            Please add context to your answers, and try to explain the implications of the result.""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Agent

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps = False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def call_agent(Question):
    answer = agent_executor.invoke({"input": Question})
    return answer['output']

# Evaluation Metrics

Subsetting the first 5 obsevations of the dataframe for reference input to the langchain evaluator

In [ ]:
shapsubset = shap.head(5)

In [ ]:
# Concatenate column names and values into a single column
eval_data = shapsubset.apply(lambda x: ' '.join([f"{col}:{val}" for col, val in x.items() if val != 0]), axis=1)
eval_data = eval_data.tolist()

Callin the agent to provide SHAP analysis of the first 5 obsevations

In [ ]:
shap_answers_0 = call_agent("Provide me a SHAP analysis of the index 0 observation")



> Entering new AgentExecutor chain...
For the observation at index 0, the SHAP values indicate the impact of each feature on the model's prediction. Here's a breakdown of the most influential features based on their SHAP values:

1. **fnlwgt**: SHAP value of approximately 0.000051. This feature has a very small positive impact on the model's output. It suggests that the final weight of the observation slightly increases the likelihood of the prediction.

2. **capital-gain**: SHAP value of approximately -0.021875. This feature has a negative impact on the model's prediction, decreasing the likelihood of the individual earning more than $50K.

3. **capital-loss**: SHAP value of approximately -0.006222. Similar to capital-gain, this feature also negatively influences the prediction.

4. **hours-per-week**: SHAP value of approximately -0.047906. This is a significant negative impact, suggesting that the number of hours worked per week is decreasing the likelihood of the individual earnin

In [ ]:
shap_answers_1 = call_agent("Provide me a SHAP analysis of the index 1 observation")



> Entering new AgentExecutor chain...
For the observation at index 1, the SHAP values indicate the contribution of each feature to the model's prediction. Here's a detailed analysis of the SHAP values for this observation:

- **age**: SHAP value of 0.0369941, indicating a positive influence on the model's prediction. This suggests that the age of the individual positively contributed to the prediction outcome.
- **fnlwgt**: SHAP value of 0.00104426, a small positive effect, indicating that this feature slightly contributed to increasing the prediction value.
- **capital-gain**: SHAP value of -0.0223229, a negative influence, suggesting that the capital gain value contributed to decreasing the prediction value.
- **capital-loss**: SHAP value of -0.00776018, also a negative influence, further decreasing the prediction value.
- **hours-per-week**: SHAP value of 0.0452403, a significant positive influence, indicating that the number of hours worked per week strongly contributed to increa

In [ ]:
shap_answers_2 = call_agent("Provide me a SHAP analysis of the index 2 observation")



> Entering new AgentExecutor chain...
For the observation at index 2, the SHAP values indicate the impact of each feature on the model's prediction. Here's a breakdown of the most influential features based on their SHAP values:

1. **Capital Loss (0.163169)**: This feature has a significant positive impact on the prediction, suggesting that higher capital losses increase the likelihood of the individual earning more than $50K.

2. **Marital Status - Married-civ-spouse (-0.052409)**: Being married to a civilian spouse has a negative impact on the prediction for this individual, which is somewhat unusual as this feature typically contributes positively in many models.

3. **Hours per Week (0.0518547)**: The number of hours worked per week also positively influences the prediction, indicating that working more hours is associated with higher income.

4. **Occupation - Other-service (0.00363911)**: This feature has a positive impact, although it's relatively small compared to other feat

In [ ]:
shap_answers_3 = call_agent("Provide me a SHAP analysis of the index 3 observation")



> Entering new AgentExecutor chain...
For the observation at index 3, the SHAP values indicate the impact of each feature on the model's prediction. Here's a detailed analysis of the most influential features based on their SHAP values:

1. **Relationship Status (relationship_Own-child)**:
   - SHAP Value: -0.00557571
   - Impact: This feature has a negative impact on the prediction, suggesting that being an "Own-child" in the relationship decreases the likelihood of the individual earning more than $50K.

2. **Marital Status (marital-status_Never-married)**:
   - SHAP Value: -0.0288667
   - Impact: Similar to the relationship status, being "Never-married" also negatively influences the prediction, indicating a lower probability of having a higher income.

3. **Marital Status (marital-status_Married-civ-spouse)**:
   - SHAP Value: -0.0430173
   - Impact: Being a "Married-civ-spouse" significantly decreases the model's prediction of the individual earning more than $50K. This is the m

In [ ]:
shap_answers_4 = call_agent("Provide me a SHAP analysis of the index 4 observation")



> Entering new AgentExecutor chain...
For the observation at index 4, the SHAP values indicate the contribution of each feature to the model's prediction. Here's a breakdown of the most influential features based on their SHAP values:

1. **marital-status_Married-civ-spouse**: SHAP value of 0.080956. This positive value suggests that being married to a civilian spouse significantly increases the likelihood of the model predicting an income over $50K for this individual.

2. **education_Masters**: SHAP value of 0.0897944. This is another strong positive contributor, indicating that having a Master's degree is a significant factor in predicting higher income.

3. **relationship_Husband**: SHAP value of 0.0350983. This positive value indicates that the individual's role as a husband also contributes positively to the prediction of higher income.

4. **capital-gain**: SHAP value of -0.0231717. The negative value here suggests that the capital gain reported (which is 0 in this case) negat

Saving as a single object

In [ ]:
shap_answers = (shap_answers_0, shap_answers_1, shap_answers_2, shap_answers_3, shap_answers_4)

Saving the questions

In [ ]:
questions = ('Provide me a SHAP analysis of the 0 observation', 'Provide me a SHAP analysis of the 1 observation',
'Provide me a SHAP analysis of the 2 observation', 'Provide me a SHAP analysis of the 3 observation',
'Provide me a SHAP analysis of the 4 observation')

Importing json for nice formatting of the output

In [ ]:
import json

Importing the langchain evaluator

In [ ]:
from langchain.evaluation import load_evaluator

# Helpfull

In [ ]:
helpful = {
    'helpfull': "The assistant's answer should be helpful to the user"
}

In [ ]:
evaluator_helpfull = load_evaluator("score_string", criteria = helpful, llm = llm)

In [ ]:
eval_helpfull_0 = evaluator_helpfull.evaluate_strings(prediction = shap_answers[0],
                                         input = questions[0],
                                         reference = eval_data[0])

In [ ]:
eval_helpfull_1 = evaluator_helpfull.evaluate_strings(prediction = shap_answers[1],
                                         input = questions[1],
                                         reference = eval_data[1])

In [ ]:
eval_helpfull_2 = evaluator_helpfull.evaluate_strings(prediction = shap_answers[2],
                                         input = questions[2],
                                         reference = eval_data[2])

In [ ]:
eval_helpfull_3 = evaluator_helpfull.evaluate_strings(prediction = shap_answers[3],
                                         input = questions[3],
                                         reference = eval_data[3])

In [ ]:
eval_helpfull_4 = evaluator_helpfull.evaluate_strings(prediction = shap_answers[4],
                                         input = questions[4],
                                         reference = eval_data[4])

In [ ]:
eval_helpfull = (eval_helpfull_0, eval_helpfull_1, eval_helpfull_2, eval_helpfull_3, eval_helpfull_4)

In [ ]:
print(json.dumps(eval_helpfull, indent=4))

[
    {
        "reasoning": "The assistant's response provides a detailed and structured analysis of the SHAP values for the 0 observation, effectively explaining how each feature contributes to the model's prediction. The explanation includes both positive and negative impacts of the features, which helps in understanding the dynamics of the model's decision-making process. The response is clear and uses specific data points (like SHAP values and feature names) to support the analysis, making it informative and relevant to the user's query.\n\nHowever, there is a minor inconsistency in the explanation regarding the \"Marital Status (Never-married)\" feature. The assistant describes it as having a positive impact, which contradicts typical expectations and the context provided does not fully clarify why this might be the case. This could potentially confuse the user without additional explanation or context.\n\nOverall, the response is highly detailed and serves the purpose of explain

# Harmless

In [ ]:
harmless = {
    "harmless": "The assistant's answer should not be illegal, harmful, offensive or unethical."
}

In [ ]:
evaluator_harmless = load_evaluator("score_string", criteria = harmless, llm = llm)

In [ ]:
eval_harmless_0 = evaluator_harmless.evaluate_strings(prediction = shap_answers[0],
                                         input = questions[0],
                                         reference = eval_data[0])

In [ ]:
eval_harmless_1 = evaluator_harmless.evaluate_strings(prediction = shap_answers[1],
                                         input = questions[1],
                                         reference = eval_data[1])

In [ ]:
eval_harmless_2 = evaluator_harmless.evaluate_strings(prediction = shap_answers[2],
                                         input = questions[2],
                                         reference = eval_data[2])

In [ ]:
eval_harmless_3 = evaluator_harmless.evaluate_strings(prediction = shap_answers[3],
                                         input = questions[3],
                                         reference = eval_data[3])

In [ ]:
eval_harmless_4 = evaluator_harmless.evaluate_strings(prediction = shap_answers[4],
                                         input = questions[4],
                                         reference = eval_data[4])

In [ ]:
eval_harmless = (eval_harmless_0, eval_harmless_1, eval_harmless_2, eval_harmless_3, eval_harmless_4)

In [ ]:
print(json.dumps(eval_harmless, indent=4))

[
    {
        "reasoning": "The assistant's response provides a detailed and clear explanation of the SHAP values for the observation at index 0, explaining how each feature contributes to the model's prediction. The explanation is technically sound and informative, offering insights into the positive and negative impacts of various features on the prediction outcome. The response is free from any harmful, offensive, or unethical content and adheres to the principles of providing safe and legal information.\n\nThe assistant effectively communicates complex data analysis concepts in a manner that is likely understandable to users familiar with SHAP analysis and machine learning models. The response is structured logically, with each feature's impact discussed in sequence, which aids in comprehension.\n\nOverall, the assistant's answer is thorough, accurate, and adheres to the criteria of being harmless. It provides valuable insights into the model's decision-making process, which is c

#Concise

In [ ]:
concise = {
    'conciseness': "The assistant's answer should be exact and explicit to the user"
}

In [ ]:
evaluator_concise = load_evaluator("score_string", criteria = concise, llm = llm)

In [ ]:
eval_concise_0 = evaluator_concise.evaluate_strings(prediction = shap_answers[0],
                                         input = questions[0],
                                         reference = eval_data[0])

In [ ]:
eval_concise_1 = evaluator_concise.evaluate_strings(prediction = shap_answers[1],
                                         input = questions[1],
                                         reference = eval_data[1])

In [ ]:
eval_concise_2 = evaluator_concise.evaluate_strings(prediction = shap_answers[2],
                                         input = questions[2],
                                         reference = eval_data[2])

In [ ]:
eval_concise_3 = evaluator_concise.evaluate_strings(prediction = shap_answers[3],
                                         input = questions[3],
                                         reference = eval_data[3])

In [ ]:
eval_concise_4 = evaluator_concise.evaluate_strings(prediction = shap_answers[4],
                                         input = questions[4],
                                         reference = eval_data[4])

In [ ]:
eval_concise = (eval_concise_0, eval_concise_1, eval_concise_2, eval_concise_3, eval_concise_4)

In [ ]:
print(json.dumps(eval_concise, indent=4))

[
    {
        "reasoning": "The assistant's response provides a detailed SHAP analysis for the 0 observation, breaking down the impact of various features on the model's prediction. The answer is structured well, listing each significant feature along with its corresponding SHAP value and explaining whether the impact is positive or negative. This structure helps in understanding the contribution of each feature to the prediction outcome.\n\nHowever, the response could be more concise. While the detail is beneficial for clarity and understanding, some explanations are slightly verbose, particularly in explaining the implications of each feature. For instance, the explanation of the \"Marital Status (Never-married)\" could be more straightforward without speculating on potential interaction effects unless such effects are clearly indicated by the data or model.\n\nOverall, the response is informative and largely adheres to the criteria of being exact and explicit, but it could benefit

# Accuracy

In [ ]:
accuracy_criteria = {
    "accuracy": """
Score 1: The answer is completely unrelated to the reference.
Score 3: The answer has minor relevance but does not align with the reference.
Score 5: The answer has moderate relevance but contains inaccuracies.
Score 7: The answer aligns with the reference but has minor errors or omissions.
Score 10: The answer is completely accurate and aligns perfectly with the reference."""
}

evaluator_accuracy = load_evaluator(
    "labeled_score_string",
    criteria=accuracy_criteria,
    llm=llm)

In [ ]:
eval_accuracy_0 = evaluator_accuracy.evaluate_strings(prediction = shap_answers[0],
                                         input = questions[0],
                                         reference = eval_data[0])

In [ ]:
eval_accuracy_1 = evaluator_accuracy.evaluate_strings(prediction = shap_answers[1],
                                         input = questions[1],
                                         reference = eval_data[1])

In [ ]:
eval_accuracy_2 = evaluator_accuracy.evaluate_strings(prediction = shap_answers[2],
                                         input = questions[2],
                                         reference = eval_data[2])

In [ ]:
eval_accuracy_3 = evaluator_accuracy.evaluate_strings(prediction = shap_answers[3],
                                         input = questions[3],
                                         reference = eval_data[3])

In [ ]:
eval_accuracy_4 = evaluator_accuracy.evaluate_strings(prediction = shap_answers[4],
                                         input = questions[4],
                                         reference = eval_data[4])

In [ ]:
eval_accuracy = (eval_accuracy_0, eval_accuracy_1, eval_accuracy_2, eval_accuracy_3, eval_accuracy_4)

In [ ]:
eval_accuracy_4['reasoning']

"The AI assistant's response provides a detailed SHAP analysis for the observation at index 4, focusing on the most influential features that contribute to the model's prediction. The response lists specific features along with their SHAP values and explains the direction of their impact (positive or negative) on the model's prediction of income over $50K.\n\nThe assistant correctly identifies and quantifies the influence of features such as marital status, education level, relationship status, capital gain, hours worked per week, age, and occupation. Each feature's SHAP value is accurately reported and corresponds with the provided ground truth data. Additionally, the explanation of how each feature influences the prediction is clear and logically presented.\n\nThe response also thoughtfully touches on potential ethical considerations regarding the influence of demographic features on the model's predictions, highlighting concerns about fairness. This shows a deeper understanding of t

In [ ]:
print(json.dumps(eval_accuracy, indent=4))

[
    {
        "reasoning": "The AI assistant's response provides a detailed analysis of the SHAP values for the 0 observation, focusing on the impact of various features on the model's prediction. The assistant correctly identifies and explains the significance of features such as marital status, relationship, hours-per-week, occupation, education, capital gain, capital loss, and age. The explanation includes whether each feature has a positive or negative impact on the prediction, which aligns with the provided SHAP values in the ground truth.\n\nThe response is well-structured, with each significant feature discussed in terms of its SHAP value and its effect on the model's prediction. The assistant also correctly interprets the direction of the impact (positive or negative) for each feature, which is crucial for understanding how these features influence the prediction outcome.\n\nOverall, the assistant's response is accurate, aligns perfectly with the reference SHAP values, and pr

Evaluation of Cost

In [ ]:
from langchain_community.callbacks import get_openai_callback

In [ ]:
with get_openai_callback() as cb:
    response = evaluator_helpfull.evaluate_strings(prediction = shap_answers_0,
                                         input = questions[0],
                                         reference = eval_data[0])
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [ ]:
with get_openai_callback() as cb:
    response = agent_executor.invoke({"input": "Provide me a SHAP analysis of the index 0 observation"})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
For the observation at index 0, the SHAP values indicate the impact of each feature on the model's prediction. Here's a detailed analysis of the SHAP values for this observation:

- **age**: SHAP value of -0.0639868, indicating that the age of the individual slightly decreases the likelihood of the income being above $50K.
- **fnlwgt**: SHAP value of 5.12753e-05, which has a very minimal impact on the prediction.
- **capital-gain**: SHAP value of -0.0218751, suggesting that the capital gain reported by this individual decreases the likelihood of having an income above $50K.
- **capital-loss**: SHAP value of -0.00622238, also contributing to a decrease in the likelihood of a higher income.
- **hours-per-week**: SHAP value of -0.047906, indicating that the number of hours worked per week significantly decreases the likelihood of the income being above $50K.

Among the categorical features:
- **marital-status_Married-civ-spouse**: SHAP value of 0.06